In [ ]:
%matplotlib inline
from rdkit import Chem
from rdkit.Chem import AllChem
import nglview
import csv
import numpy as np
import matplotlib.pyplot as plt

### As a starting point, you have been given a list of molecules in SMILES format containing compounds taken from the WHO (World Health Organization) Essential Medicine List. 

Many, but not all of the essential medicines are small molecules. Roughly speaking, a small molecule is a molecule of relatively low molecular weight - a commonly cited value is < 900 daltons - in order to distinguish them from larger molecules such as proteins and polymers.

`WHO_EML_SMILES.csv` is a text file, where on each line there are two values separated by a comma. The first value is the name of the drug and the second value contains the 2D molecular structure in SMILES format. These SMILES strings were generated as part of a [research paper](https://doi.org/10.1039/C9RE00348G) and kindly provided by the authors, Klavs Jensen and Hanyu Gao.

We will begin by using the `csv` module to read in the file. As we can see here, the csv.reader function returns an iterable object containing the contents of the file.

In [ ]:
with open('WHO_EML_SMILES.csv') as csv_file:
    reader = csv.reader(csv_file)
    for line in reader:
        print(line)

Take abacavir for example, the first line in the file. This is a medication used to treat HIV/AIDS. To start working with this molecule, we could build the structure in Avogadro, but a more systematic way would be to create a molecule object directly from the SMILES string.  The RDKit toolkit allows us to do this.

In [ ]:
# Generate an RDKit molecule object from a SMILES string.
# The example uses abacavir taken from the above printout.
m1 = Chem.MolFromSmiles('Nc1nc(NC2CC2)c3ncn([C@@H]4C[C@H](CO)C=C4)c3n1')

In [ ]:
# This is how the RDKit molecule object is represented - as a 2D drawing.
m1

In [ ]:
# Starting from SMILES, there are no 3-D coordinates and they need to be generated.
# This is how RDKit generates 3-D conformers for a molecule.
# Under the hood, it uses a method called ETDKG to do this.
AllChem.EmbedMolecule(m1)

In [ ]:
# Now we can print out the 3D coordinates. Note that hydrogen atoms are missing.
print(Chem.MolToXYZBlock(m1))

In [ ]:
# nglview has a nice tool for allowing us to view the 3D coordinates as well.
nglview.show_rdkit(m1)

In [ ]:
# Note that the structure has no hydrogens. This will interfere with quantum calcs
# and possibly even with conformer generation (though it didn't this time).
# Hydrogens need to be added explicitly to a structure in RDKit.
m1h = Chem.AddHs(m1)
m1h

In [ ]:
# The 2D structure doesn't make sense but that's because we need to re-run the conformer generation
# to place the hydrogen atoms in the right places. In the future, you could run the "Chem.AddHs()"
# immediately after creating the molecule from SMILES.
AllChem.EmbedMolecule(m1h)
m1h

In [ ]:
# Ah, much better
print(Chem.MolToXYZBlock(m1h))
nglview.show_rdkit(m1h)

The power of cheminformatic tools becomes apparent when you want to work with big batches of molecules. For example we can create RDKit molecule objects from each molecule in the file:

In [ ]:
WHO_Molecules = {}
with open('WHO_EML_SMILES.csv') as csv_file:
    reader = csv.reader(csv_file)
    for line in reader:
        key = line[0]
        smi = line[1]
        WHO_Molecules[key] = Chem.MolFromSmiles(smi)

Running the above command produced a few warnings for me, which is to be expected when working with large datasets, but there were no errors.

Now that we have a list of Molecule objects, we could calculate some properties.  For example, let's calculate the molecular weight of all the compounds and make a histogram.  The correct API call to use can be obtained by Google searching "rdkit molecular weight". It gives us a function `rdkit.Chem.Descriptors.ExactMolWt` which takes a molecule object as input.

In [ ]:
# For reasons unknown, we had to important ExactMolWt explicitly.
from rdkit.Chem.Descriptors import ExactMolWt
MolWts = []
for molname, molobj in WHO_Molecules.items():
    MolWts.append(ExactMolWt(molobj))
plt.hist(MolWts)

The above histogram was somewhat useful but we didn't get much info on the distribution of molecular weights other than that they are mostly under 1000.  Apparently the list also contains just a few molecules with even larger weights.  How about we plot a histogram with just the weights under 1000?

In [ ]:
MolWts = np.array(MolWts)
plt.hist(MolWts[np.where(MolWts<1000)])

Great, that's a bit more informative.  We could also count the number of carbon atoms in each molecule and see if it's correlated with the molecular weight. After a bit of experimentation I figured out how to get the atomic symbols for an example molecule:

In [ ]:
mol_a = WHO_Molecules['abacavir']
for atom in mol_a.GetAtoms():
    print(atom.GetSymbol())

In [ ]:
# In Python 3.6 onwards (we are using 3.7 I think), dictionary keys are ordered.
# In past versions and possibly other Python implementations, the keys may not be ordered.
# Therefore, it is a good idea to make a sorted list of keys before making element-wise
# comparisons between properties.
NumCarbons = []
MolWts = []
MolNames = sorted(list(WHO_Molecules.keys()))
for molname in MolNames:
    molobj = WHO_Molecules[molname]
    MolWts.append(ExactMolWt(molobj))
    num_C = 0
    for atom in molobj.GetAtoms():
        if atom.GetSymbol() == 'C':
            num_C += 1
    NumCarbons.append(num_C)
plt.scatter(MolWts, NumCarbons)
# I set the plot limits to focus on the MW < 1000 molecules.
plt.xlim(0, 1000)
plt.ylim(-2, 50)

One of the nifty features of RDKit is that it can draw multiple molecules in a grid image.  By specifying the legends keyword argument, we can put a text label beneath each molecule.  The WHO essential medicine list has too many molecules to draw in a single image, so the following command draws only a few dozen.

In [ ]:
Chem.Draw.MolsToGridImage(list(WHO_Molecules.values()), legends=list(WHO_Molecules.keys()))

One of the most powerful tools in cheminformatics is the ability to "search" molecules for chemical patterns.  The SMARTS language is the language for specifying a chemical pattern.  The molecule represented by a SMILES string is usually, but not always, matched by the same string when used as a SMARTS.  Here we will create a RDKit molecule from a SMARTS string for a benzene ring, and apply it as a pattern to a molecule.

In [ ]:
benz = Chem.MolFromSmarts('c1ccccc1')
benz

The depiction of the SMARTS benzene is slightly different from the SMILES. Now we can call the `mol.GetSubstructMatches()` method, passing the benzene SMARTS molecule as the argument, and it will return the atom indices for the matching atom groups.

In [ ]:
WHO_Molecules['amodiaquine'].GetSubstructMatches(benz)

RDKit also conveniently highlights the portions of the molecule corresponding to a positive match.

In [ ]:
WHO_Molecules['amodiaquine']

Another great feature of RDKit is the ability to find common substructures in molecules. This can be very important for chemistry research because some compounds are "relatives" of one another in the sense that they have similar functions, or have the same molecular "core", or both. For example let's take the antiviral drugs abacavir and aciclovir. We can see that there is a fused ring structure corresponding to a purine that is shared by both. 

In [ ]:
abac = WHO_Molecules['abacavir']
acic = WHO_Molecules['aciclovir']
Chem.Draw.MolsToGridImage([abac,acic], legends=['abacavir','aciclovir'])

Let's see if RDKit can find the common substructure for us. This requires importing a submodule of RDKit called `rdFMCS`, where the MCS stands for "Maximum Common Substructure".

In [ ]:
from rdkit.Chem import rdFMCS
# The return value of FindMCS() is a MCSResult object.
# The MCSResult can be converted to a SMARTS string, which could then be converted to a molecule object.
# I'm not sure why RDKit doesn't simply return the molecule object directly, but it's not so bad.
mcs = rdFMCS.FindMCS([abac,acyc])
mcsMol = Chem.MolFromSmarts(mcs.smartsString)
mcsMol

Now that we have the maximum common structure as a Molecule object, we can draw the two molecules together with the common structure highlighted.

In [ ]:
abac_matches = abac.GetSubstructMatch(mcsMol)
acic_matches = acic.GetSubstructMatch(mcsMol)
Chem.Draw.MolsToGridImage([abac,acic], legends=['abacavir','aciclovir'], highlightAtomLists=[abac_matches, acic_matches])

With these methods in hand, you should now be able to compare pairwise similarities between long lists of molecules, or mine a large dataset of molecules for those that contain functional groups of interest.  The possibilities are endless!

#### So far, we have only worked on the SMILES strings. Now let's see how we can incorporate some experimental data. 

There are lots of kinds of experimental data, and here we will show how to get some crystal structures into the notebook.  This will require doing some work outside of the notebook as well.

First, pick a SMILES string of interest. We will look at clomipramine, whose SMILES string is CN(C)CCCN1c2ccccc2CCc3ccc(Cl)cc13 .  You can get the SMILES strings from the .csv file either by opening it directly as a spreadsheet in Excel or other software, or by printing the keys of the `WHO_Molecules` dictionary from above.

Connect to "CCDC Access Structures" at the link: https://www.ccdc.cam.ac.uk/structures/ .  Make sure you are on campus or using the [library VPN](https://www.library.ucdavis.edu/service/connect-from-off-campus/), otherwise you will hit a paywall at the next step.  

![Structure search](https://i.imgur.com/dkbIz0t.png)

Click on the "Structure Search" tab.  Click the "Advanced ↓" button at the lower left.  Click the "Substructure" radio button next to "Match condition:", and paste the SMILES string for your molecule into the box that says SMARTS.  (SMARTS is a language for searching for molecular structures, and generally a SMILES string is a valid SMARTS string).  You might get an error for SMILES strings that contain "@" characters that specify stereochemistry.  In these cases, remove the "@" signs from your string using a text editor, or a command line tool such as sed.

![Search results](https://i.imgur.com/q1k4J2Z.png)

After the search, you should see several results pop up.  Crystal structues often contain more than just the molecule of interest.  For example, the first entry that popped up in my search, which had the identifier BUXKIR, had the molecule of interest inside of a cyclodextrin ring.  Some molecules may be similar but not identical to the molecule you're searching for.  Look for a structure where the details include the name of the drug.  For example, CIMPRA lists "Chlorimipramine hydrochloride" under Synonyms, and a quick Wikipedia search shows that Chlorimipramine and clomipramine are synonymous.  That should be close enough for us.  Download the CIF file to your hard drive by clicking Download -> Download Current Entry.  Move the CIF file to the same folder where you have your notebook. 

Before you can load a structure into RDKit, another cheminformatic tool called OpenBabel is needed to convert the .cif file into a file format that RDKit can read (here we will use .mol2).  In the folder where you have your notebook and the .mol2 file, run OpenBabel in your terminal (not in the notebook!) as follows:

```
(che155) $ obabel -icif 1125716.cif -omol2 -O 1125716.mol2
1 molecule converted
```

Now you should be able to load the .mol2 file into your notebook.  The default behavior is to remove hydrogen atoms from the structure, so we pass in the keyword argument to keep hydrogens ([see RDKit documentation](https://www.rdkit.org/docs/source/rdkit.Chem.rdmolfiles.html#rdkit.Chem.rdmolfiles.MolFromMol2File).)

In [ ]:
mol_exp = Chem.MolFromMol2File('1125716.mol2', removeHs=False)

We can also use NGLView to show the 3D structure of the molecule here:

In [ ]:
nglview.show_rdkit(mol_exp)

An interesting question to ask is, how well does the computationally generated conformation agree with the experimental one?  This touches on the cutting-edge field of crystal structure prediction, which is very important to industry as well as basic science.

To look at this, we can create another RDKit molecule object directly from the SMILES string with no additional experimental data, generate a number of conformers, and compare the results to the above.  The API call `AllChem.EmbedMultipleConfs` is how we ask RDKit to generate multiple conformers for a molecule ([see RDKit documentation](https://www.rdkit.org/docs/source/rdkit.Chem.rdDistGeom.html#rdkit.Chem.rdDistGeom.EmbedMultipleConfs).)

In [ ]:
mol_smi = Chem.MolFromSmiles('CN(C)CCCN1c2ccccc2CCc3ccc(Cl)cc13')
mol_smi_h = AllChem.AddHs(mol_smi)
# The random seed argument is added so that multiple runs will give the same result.
AllChem.EmbedMultipleConfs(mol_smi_h, 10, randomSeed=8282)

Using NGLView we can visualize each 3D conformation.  The keyword argument for controlling which conformer to show is `conf_id` (I figured this out using `help(nglview.show_rdkit)`).  I don't know whether it's possible change the conformer being viewed with a slider.

In [ ]:
nglview.show_rdkit(mol_smi_h, conf_id=3)

A powerful feature of RDKit is the ability to perform structure alignment.  This is a calculation in which the structures of two conformations are compared by applying a translation and rotation to one confomation, called the probe, in order to minimize the distances to a reference conformation.  After applying the rotation and translation, the remaining Euclidean distances between the atoms of the probe and reference structure are a measure of the "internal" structural differences due to differences in bond lengths, etc.  By taking the RMS of all the displacements for each atom, one can calculate the "RMSD" which is a single number that measures the difference between two structures.

In order to rotate two structures to minimize the distances between corresponding atoms, both structures need to have the same atom ordering, which is actually not true for our experimental structure and the generated one.  What's more, the atoms of the probe and reference structure need to be the same, and this is not strictly true for our example either, because our experimental structure is actually a hydrochloride salt, containing an extra HCl that the generated structure doesn't have.  RDKit takes care of both of these problems by finding a substructure that the probe and reference structures have in common; the substructure matching also generates a mapping of the atom orderings between one structure and the other, so we are able to get a result without worryinng.

In [ ]:
for i in range(mol_smi_h.GetNumConformers()):
    print(Chem.rdMolAlign.AlignMol(mol_smi_h, mol_exp, i))

The interpretation of RMSD depends on the situation, but here we use a very rough rule of thumb that a RMSD value of 1 Angstrom or less indicates a high level of agreement for molecular structures.  Above we see that conformer number 6 has the best level of agreement with experiment.  Let's plot them together to look at how close the structures are.

Using the `update_representation()` method, we are able to change the color of each structure being shown (called the "component"). The structure shown in transparent red is the experimental structure.

In [ ]:
view = nglview.show_rdkit(mol_smi_h, conf_id=6)
view.add_component(mol_exp)
view.update_representation(component=1, color='red', opacity=0.5)
view